In [28]:
from collections import deque
import numpy as np
import glob

"""
Zvolil jsem průchod BFS (Breadth-First Search) pro hledání cesty v bludišti, protože
BFS je vhodný pro hledání nejkratší cesty v neorientovaných grafech a mřížkách.
Zároveň jsme BFS implementovali v předmětu Algoritmy 1 a vím o něm více 
než o Dijkstrově algoritmu...
"""

# Předpokládám, že tento kód bude spuštěn v adresáři, kde se nachází pouze CSV soubor s maticí
# Najde všechny CSV soubory v aktuálním adresáři 
# (dopředu nevím, jaký bude název csv souboru, proto to řeším takto)
csv_files = glob.glob("*.csv")

if csv_files:
    file_path = csv_files[0] # vezme první nalezený CSV soubor
    data = np.loadtxt(file_path, delimiter=",")
    # vznikne binary matrix: 0 → True, 1 → False
    binary_matrix = (data == 0)

else:
    print("Nebyl nalezen žádný CSV soubor.")
    exit()



def save_path_to_csv(path_map, filename="vystup.csv"):
    # True → 0, False → 1
    output_matrix = np.where(path_map, 0, 1)
    
    # Uloží do CSV se separátorem ","
    np.savetxt(filename, output_matrix, fmt="%d", delimiter=",")
    print(f"Výstupní matice byla uložena jako '{filename}'.")


def get_neighbors(i, j, n):
    neighbors = []
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # nahoru, dolů, vlevo, vpravo

    for di, dj in directions:
        ni, nj = i + di, j + dj
        if 0 <= ni < n and 0 <= nj < n: # zkontrolujeme, jestli je potenciální soused v rámci matice 
            neighbors.append((ni, nj))  # přidáme souseda jako dvojici (řádek, sloupec)

    return neighbors


def show_path(n, parent_map, end):
    path_map = np.full((n, n), False, dtype=bool)
    # vytvoříme matici pro zobrazení cesty, kde True znamená, že buňka je součástí cesty
    path_steps = [] # list pro uchování souřadnic buněk, které jsou součástí cesty
    current = end
    p = 0 # počet buněk, které jsou součástí cesty
    # (p má využití pří vytváření bludiště)
    while current is not None:
        p += 1
        path_map[current] = True
        path_steps.append(current)
        current = parent_map[current]

    return path_map, p, path_steps[::-1]
    # vrátíme matici cesty, počet buněk na cestě
    # a zpětně seznam souřadnic buněk, které jsou součástí cesty



def solve(matrix):
    n = matrix.shape[0]
    state_map = np.full(matrix.shape, "unknown", dtype=object) 
    # počáteční stav všech buněk je "unknown", v průběhu prohledávání se mění na "discovered" a "finished"
    parent_map = np.full(matrix.shape, None, dtype=object)
    # parent_map slouží k uchování "předků", abychom mohli sledovat cestu zpět.
    # pro konkrétní buňky bude obsahovat dvojice (i, j) pro jejich předka.
    # dtype=object -> NumPy musí vědět, že ty položky jsou obecné Python objekty.
    # položky jsou buď dvojice (i, j) nebo None (pro počáteční buňku) pro parent_map
    # a "unknown", "discovered", "finished" pro state_map
    queue = deque()

    state_map[0, 0] = "discovered"
    parent_map[0, 0] = None # počáteční buňka nemá žádného předka
    queue.append((0, 0))  # algoritmus vždy začneme v levém horním rohu

    while queue:
        # klasický bsf algoritmus
        i, j = queue.popleft()
        
        for ni, nj in get_neighbors(i, j, n):
            if (ni, nj) == (n-1, n-1):
                # pokud jsme dosáhli pravého dolního rohu, můžeme skončit
                parent_map[ni, nj] = (i, j)
                # p teď nepotřebujeme, proto _
                path_map, p, path_steps = show_path(n, parent_map, (n-1, n-1))
                return path_map, p, path_steps
            if state_map[ni, nj] == "unknown" and matrix[ni, nj]:
                # pokud je buňka neznámá a je průchozí (True), přidáme ji do fronty
                state_map[ni, nj] = "discovered"
                parent_map[ni, nj] = (i, j)
                queue.append((ni, nj))
        state_map[i, j] = "finished"

    # Pokud se sem dostaneme, žádná cesta neexistuje
    print("Cesta nebyla nalezena.")
    return None


solve_path, _, _ = solve(binary_matrix)

if solve_path is not None:
    save_path_to_csv(solve_path)
    # Pokud je cesta nalezena, uložíme ji do CSV souboru









Výstupní matice byla uložena jako 'vystup.csv'.


In [ ]:
import random
import numpy as np


def rand_dir(i, j, n):
    # Funkce pro generování náhodného směru v bludišti
    neighbors = get_neighbors(i, j, n)
    cur_dir = random.choice(neighbors)
    return cur_dir

# n - velikost matice (n x n)
def create_simple_tem(n):
    template = np.ones((n, n), dtype=int)

    mid = n // 2  # prostřední řádek

    for i in range(mid + 1):
        template[i, 0] = 0  # cesta dolů doprostřed v levém sloupci

    for j in range(n):
        template[mid, j] = 0  # cesta doprava

    for i in range(mid + 1, n):
        template[i, n - 1] = 0 # cesta dolů v pravém sloupci

    return template


# z - velikost "zigzag" úseček
def create_zigzag_tem(n, z):
    template = np.ones((n, n), dtype=int)

    i = 0
    j = 0

    # hlavní zigzag část
    while i + z < n and j + z < n:
        # svislý úsek dolů
        for k in range(z):
            template[i + k, j] = 0
        i += z - 1  # končíme dole

        # vodorovně doprava
        for l in range(1, z):
            template[i, j + l] = 0
        j += z - 1  # končíme vpravo

        # příprava na další "zig zag"
        
    template[i, j] = 0
    
    # dokončení do pravého dolního rohu
    while (i, j) != (n - 1, n - 1):
        
        if i < n - 1:
            i += 1
        template[i, j] = 0
        if j < n - 1:
            j += 1
        template[i, j] = 0
            
    return template


# f - fraction - určuje, do jakého zlomu bude matice rozdělena
def create_best_tem(n, f):
    template = np.ones((n, n), dtype=int)

    mid = n // 2
    fraction = n // f

    # 1. Dolů po levém okraji
    for i in range(mid + 1):
        template[i, 0] = 0

    # 2. Doprava ve středu
    for j in range(mid + 1):
        template[mid, j] = 0

    # 3. Nahoru do zlomu ve sloupci mid
    for i in range(mid - 1, fraction - 1, -1):
        template[i, mid] = 0

    # 4. Doprava z mid na n - fraction v řádku fraction
    for j in range(mid + 1, n - fraction):
        template[fraction, j] = 0

    # 5. Dolů vpravo – sloupec n - fraction
    for i in range(fraction + 1, n - fraction):
        template[i, n - fraction - 1] = 0

    # 6. Doleva – v řádku n - fraction
    for j in range(n - fraction - 2, mid - 1, -1):
        template[n - fraction - 1, j] = 0

    # 7. Dolů středem ke spodnímu řádku
    for i in range(n - fraction, n):
        template[i, mid] = 0

    # 8. Doprava do pravého dolního rohu9
    for j in range(mid + 1, n):
        template[n - 1, j] = 0

    return template


# tato turbo funkce není vždy úspěšná, ale snaží se vytvořit složitější bludiště
def create_turbo_tem(n):
    template = np.ones((n, n), dtype=int)
    mid = n // 2

    # výběr frakcí s dodatečnou podmínkou pro fraction3
    fraction1 = random.randint(3, 5)
    fraction2 = random.randint(6, 8)
    fraction3 = random.randint(fraction2 + 1, 12)  # zajistí, že fraction3 - 1 > fraction2

    # 1. Dolů
    for i in range(fraction2 + 1):
        template[i, 0] = 0
    cur_i = i

    # 2. Doprava
    for j in range(fraction1 + 1):
        template[cur_i, j] = 0
    cur_j = j

    # 3. Nahoru
    for i in range(cur_i, fraction3 - 1, -1):
        template[i, cur_j] = 0
    cur_i = i

    # 4. Doprava
    for j in range(cur_j + 1, cur_j + fraction1):
        template[cur_i, j] = 0
    cur_j = j

    # 5. Dolů
    for i in range(cur_i + 1, cur_i + fraction2):
        template[i, cur_j] = 0
    cur_i = i

    # 6. Doleva
    for j in range(cur_j - 1, cur_j - fraction3, -1):
        template[cur_i, j] = 0
    cur_j = j

    # 7. Dolů
    for i in range(cur_i + 1, mid + 1):
        template[i, cur_j] = 0
    cur_i = i

    # 8. Doleva
    for j in range(cur_j - 1, fraction3, -1):
        template[cur_i, j] = 0
    cur_j = j

    # 9. Dolů
    for i in range(cur_i + 1, n - fraction2):
        template[i, cur_j] = 0
    cur_i = i

    # 10. Doprava
    for j in range(cur_j + 1, cur_j + fraction3):
        template[cur_i, j] = 0
    cur_j = j

    # 11. Nahoru
    for i in range(cur_i - 1, mid, -1):
        template[i, cur_j] = 0
    cur_i = i

    # 12. Doprava
    for j in range(cur_j + 1, n):
        template[cur_i, j] = 0
    cur_j = j

    # 13. Dolů
    for i in range(cur_i + 1, n - fraction3):
        template[i, cur_j] = 0
    cur_i = i

    # 14. Doleva
    for j in range(cur_j - 1, fraction2, -1):
        template[cur_i, j] = 0
    cur_j = j

    # 15. Dolů
    for i in range(cur_i + 1, n):
        template[i, cur_j] = 0

    # 16. Doprava do pravého dolního rohu
    for j in range(cur_j + 1, n):
        template[n - 1, j] = 0

    return template

# t různých šablon pro generování bludiště

def create_maze(n, t):

    if n < 12 or n > 1000:
        print("Velikost matice musí být v rozmezí 12 až 1000.")
        return None

    # Zde zvolíme šablonu podle typu t
    if t == 1:
        maze = create_simple_tem(n)
    elif t == 2:
        maze = create_zigzag_tem(n, 3)
    elif t == 3:
        maze = create_best_tem(n, 5)
    elif t == 4:
        maze = create_turbo_tem(n)
    else:
        print("Neplatný typ šablony. Používám jednoduchou šablonu.")
        maze = create_simple_tem(n)
    
    # maze budeme teď už měnit dále, path_map se hodí k uložení cesty
    # teď potřebujeme maze dostat do formátu, který bude použitelný pro funkci solve
    # path_map: 1 → False (není průchozí), 0 → True (průchozí)

    """
    protože varianta 4 (turbo) nemusí vždy fungovat,
    vytvoříme novou šablonu pro t = 3, 
    pokud se nepodaří najít cestu pro t = 4,
    a použijeme ji pro další pokus o nalezení cesty.
    """

    converted_maze = (maze == 0)
    result = solve(converted_maze)
    if result is None:
        if t != 4:
            print("Cesta nebyla nalezena nebo má nesprávný formát.")
            return None
        else:
            maze = create_best_tem(n, 5)  # pokud šablona turbo nevyšla, vytvoříme novou šablonu
            converted_maze = (maze == 0)
            result = solve(converted_maze)
            if result is None:
                print("Cesta nebyla nalezena ani po vytvoření nové šablony.")
                return None
    # pokud je cesta nalezena, uložíme ji do proměnných
    path_map, _, win_steps = result

    # Přidáme náhodné falešné cesty do bludiště
    num_paths = n // 3 # (podle mě) optimální počet falešných cest
    path_length = n  # (opět podle mě) optimální délka falešných cest 
    opt_path_start = n // 3 # chceme, aby falešné cesty prvních n // 3 buněk vedly dál od win_steps

    # path_map využijeme pro uložení všech možných cest (včetně falešných)
    # pomocí slice ořezáváme win_steps, aby se vyhnuly okrajům
    # a získali jsme pouze vnitřní buňky, kde můžeme přidávat falešné cesty
    opt_steps = win_steps[2:-2] 
    paths = random.sample(opt_steps, num_paths)
    c = 0 # count pro počet kroků
    for (i, j) in paths:
        for _ in range(path_length):
            # pokud jsme na okraji, tak už nemůžeme pokračovat
            if (i, j) == (n - 1, n - 1):
                break
            if c == 0:
                maze[i, j] = 0 # nastavíme aktuální buňku jako průchozí
                path_map[i, j] = True # nastavíme buňku jako součást cesty
            elif c <= opt_path_start:
                # nejprve zjistíme, kteří sousedi buňky jsou součástí win_steps
                neighbors = get_neighbors(i, j, n)
                for ni, nj in neighbors:
                    if (ni, nj) not in win_steps:
                        if path_map[ni, nj] == False: # pokud není buňka součástí cesty
                            maze[ni, nj] = 0
                            path_map[ni, nj] = True # nastavíme buňku jako součást cesty
                        # zjistíme, kterým směrem jsme se posunuli 
                        # z aktuální buňky (i, j) do sousední buňky (ni, nj)
                        direction = None
                        di = ni - i
                        dj = nj - j

                        if di == -1 and dj == 0:
                            direction = (-1, 0) # nahoru
                        elif di == 1 and dj == 0:
                            direction = (1, 0) # dolů
                        elif di == 0 and dj == -1:
                            direction = (0, -1) # doleva
                        elif di == 0 and dj == 1:
                            direction = (0, 1) # doprava
                        if direction is not None:
                            # posuneme se z buňky (ni, nj) o daný směr
                            ni += direction[0]
                            nj += direction[1]
                        while c <= opt_path_start and (0 <= ni < n and 0 <= nj < n):
                            # pokud jsme v rámci matice, nastavíme buňku jako průchozí
                            maze[ni, nj] = 0
                            # posuneme se o daný směr
                            ni += direction[0]
                            nj += direction[1]
                            # zvýšíme počet kroků
                            c += 1

            # pokud už nejsme na začátku falešné cesty, můžeme nový směr dát jako random
            maze[i, j] = 0 # nastavíme aktuální buňku jako průchozí
            # náhodně zvolíme směr
            ni, nj = rand_dir(i, j, n)
            if maze[ni, nj] == 1:  # pokud je buňka volná
                maze[ni, nj] = 0  # vytvoříme falešnou cestu
            elif maze[ni, nj] == 0:  # pokud je buňka průchozí
                for _ in range(4): # tři další pokusy o nalezení průchozí buňky
                    ni, nj = rand_dir(i, j, n)
                    if maze[ni, nj] == 1:
                        # nastavíme aktuální buňku jako průchozí
                        maze[ni, nj] = 0
                        break
            i, j = ni, nj  # posuneme se na novou buňku
            
    # nakonec vytvoříme novou matici, která je nové bludiště,
    # kde 0 znamená průchozí buňku
    # a 1 znamená neprůchozí buňku
    # a uložíme ji do CSV souboru
    new_maze = (maze == 0)
    save_path_to_csv(new_maze, "generated_maze.csv")
    print("Bludiště bylo úspěšně vygenerováno a uloženo do 'generated_maze.csv'.")

create_maze(80, 4)

Výstupní matice byla uložena jako 'generated_maze.csv'.
Bludiště bylo úspěšně vygenerováno a uloženo do 'generated_maze.csv'.
